In [ ]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Patch
from matplotlib.lines import Line2D
from typing import List, Tuple, Optional
import random
import os


class Robot:
    def __init__(self, position: Tuple[float, float], direction: Tuple[float, float], robot_id: int):
        self.position = np.array(position, dtype=float)
        self.direction = np.array(direction, dtype=float)
        if np.linalg.norm(self.direction) != 0:
            self.direction = self.direction / np.linalg.norm(self.direction)
        else:
            # default direction if zero vector passed
            self.direction = np.array([1.0, 0.0])
        self.id = robot_id

        # Physical properties
        self.radius = 0.3
        self.speed = 0.05

        # Behavioral properties
        self.communication_range = 2.5
        self.avoidance_radius_illuminated = 0.51
        self.avoidance_radius_shadowed = 0.4

        # State
        self.is_illuminated = False
        self.current_avoidance_radius = self.avoidance_radius_shadowed
        self.previous_neighbor_count = 0

    def update_illumination(self, beacon_position: np.ndarray, other_robots: List['Robot']):
        """Check if robot is illuminated by beacon or shadowed by other robots"""
        self.is_illuminated = True

        for other in other_robots:
            if other.id == self.id:
                continue

            to_this = self.position - beacon_position
            to_other = other.position - beacon_position

            distance_this = np.linalg.norm(to_this)
            distance_other = np.linalg.norm(to_other)

            # if other is further from beacon than this robot, it can't shadow
            if distance_other >= distance_this:
                continue

            if distance_this > 0:
                direction = to_this / distance_this
            else:
                continue

            projection = np.dot(to_other, direction)
            closest_point_on_ray = beacon_position + direction * projection
            distance_to_ray = np.linalg.norm(other.position - closest_point_on_ray)

            # if other intersects the ray from beacon to this robot and lies between beacon & this robot
            if distance_to_ray < other.radius and 0 < projection < distance_this:
                self.is_illuminated = False
                break

        if self.is_illuminated:
            self.current_avoidance_radius = self.avoidance_radius_illuminated
        else:
            self.current_avoidance_radius = self.avoidance_radius_shadowed

    def check_neighbors(self, other_robots: List['Robot']) -> int:
        """Count robots within communication range"""
        count = 0
        for other in other_robots:
            if other.id == self.id:
                continue
            distance = np.linalg.norm(self.position - other.position)
            if distance <= self.communication_range:
                count += 1
        return count

    def detect_obstacle(self, other_robots: List['Robot']) -> Tuple[bool, Optional[np.ndarray]]:
        """Check if there's an obstacle within avoidance radius"""
        for other in other_robots:
            if other.id == self.id:
                continue
            distance = np.linalg.norm(self.position - other.position)
            if distance <= self.current_avoidance_radius:
                return True, other.position
        return False, None

    def avoid_obstacle(self, obstacle_position: np.ndarray):
        """Turn away from obstacle (opposite direction)"""
        away_vector = self.position - obstacle_position
        norm = np.linalg.norm(away_vector)
        if norm > 0:
            self.direction = away_vector / norm

    def coherence_turn(self):
        """Turn 180 degrees"""
        self.direction = -self.direction

    def random_turn(self):
        """Turn to random direction"""
        angle = random.uniform(0, 2 * np.pi)
        self.direction = np.array([np.cos(angle), np.sin(angle)])

    def move(self):
        """Move forward in current direction"""
        self.position += self.direction * self.speed


class Beacon:
    def __init__(self, position: Tuple[float, float]):
        self.position = np.array(position, dtype=float)


class Simulation:
    def __init__(self, num_robots: int, max_steps: int, world_size: Tuple[float, float],
                 initialization_mode: str = 'connected', enable_plotting: bool = False,
                 plot_interval: int = 5, plot_output_dir: str = 'plots',
                 track_metrics: bool = False):
        self.num_robots = num_robots
        self.max_steps = max_steps
        self.world_size = world_size
        self.initialization_mode = initialization_mode
        self.enable_plotting = enable_plotting
        self.plot_interval = plot_interval
        self.plot_output_dir = plot_output_dir
        self.track_metrics = track_metrics
        self.current_step = 0

        self.robots: List[Robot] = []
        self.beacon: Optional[Beacon] = None
        self.position_history = []

        # Metrics tracking (for Part B)
        self.distances_to_beacon: List[float] = []
        self.cohesion_values: List[float] = []

        self._initialize_robots()
        self._initialize_beacon()

        if self.enable_plotting:
            os.makedirs(self.plot_output_dir, exist_ok=True)
            print(f"\n✓ Plot output directory created: '{self.plot_output_dir}/'")

    def _initialize_robots(self):
        """Create robots based on initialization mode"""
        if self.initialization_mode == 'connected':
            self._initialize_robots_connected()
        elif self.initialization_mode == 'random':
            self._initialize_robots_random()
        else:
            raise ValueError(f"Unknown initialization mode: {self.initialization_mode}")

        # Initialize previous neighbor counts
        for robot in self.robots:
            robot.previous_neighbor_count = robot.check_neighbors(self.robots)

        if self.initialization_mode == 'connected':
            self._verify_connectivity()

    def _record_metrics(self):
        """Record distance to beacon and swarm cohesion"""
        positions = np.array([robot.position for robot in self.robots])
        if positions.size == 0:
            # no robots, nothing to record
            self.distances_to_beacon.append(np.nan)
            self.cohesion_values.append(np.nan)
            return

        centroid = np.mean(positions, axis=0)
        distance_to_beacon = np.linalg.norm(centroid - self.beacon.position)
        cohesion = np.mean(np.linalg.norm(positions - centroid, axis=1))
        self.distances_to_beacon.append(float(distance_to_beacon))
        self.cohesion_values.append(float(cohesion))

    def _initialize_robots_connected(self):
        """Create robots in a cluster to ensure full connectivity"""
        center_x = self.world_size[0] * 0.2
        center_y = self.world_size[1] / 2
        max_radius = 1.0

        for i in range(self.num_robots):
            angle = random.uniform(0, 2 * np.pi)
            radius = random.uniform(0, max_radius)

            x = center_x + radius * np.cos(angle)
            y = center_y + radius * np.sin(angle)

            dir_angle = random.uniform(0, 2 * np.pi)
            direction = (np.cos(dir_angle), np.sin(dir_angle))

            robot = Robot((x, y), direction, i)
            self.robots.append(robot)

    def _initialize_robots_random(self):
        """Create robots with random positions across the world"""
        for i in range(self.num_robots):
            x = random.uniform(0, self.world_size[0])
            y = random.uniform(0, self.world_size[1])
            angle = random.uniform(0, 2 * np.pi)
            direction = (np.cos(angle), np.sin(angle))

            robot = Robot((x, y), direction, i)
            self.robots.append(robot)

    def _verify_connectivity(self):
        """Verify that all robots are connected"""
        print("\nVerifying connectivity...")
        for i, robot in enumerate(self.robots):
            neighbors = robot.check_neighbors(self.robots)
            print(f"  Robot {i}: {neighbors} neighbors")

        min_neighbors = min(robot.check_neighbors(self.robots) for robot in self.robots)
        if min_neighbors == self.num_robots - 1:
            print("All robots are fully connected!")
        else:
            print(f"Warning: Minimum neighbors = {min_neighbors}, expected {self.num_robots - 1}")

    def _initialize_beacon(self, position: Tuple[float, float] = None):
        """Place beacon in the world"""
        if position is None:
            # Place beacon to the right (beacon on same level as robots)
            position = (self.world_size[0] * 0.8, self.world_size[1] / 2)
        self.beacon = Beacon(position)

    def step(self):
        """Execute one simulation time step following the paper's algorithm"""

        # 1. Update illumination status for all robots
        for robot in self.robots:
            robot.update_illumination(self.beacon.position, self.robots)

        # 2. For each robot, check conditions and execute behaviors
        # The order is: coherence > random > avoid > forward (as per state machine)
        for robot in self.robots:
            current_neighbors = robot.check_neighbors(self.robots)

            # Check coherence: if neighbors dropped below threshold
            # Using α = N-1 (all other robots) as per the paper
            if current_neighbors < self.num_robots - 1:
                robot.coherence_turn()
                robot.previous_neighbor_count = current_neighbors
                continue

            # Check random turn: if neighbors increased
            if current_neighbors > robot.previous_neighbor_count:
                robot.random_turn()
                robot.previous_neighbor_count = current_neighbors
                continue

            # Check avoidance
            has_obstacle, obstacle_pos = robot.detect_obstacle(self.robots)
            if has_obstacle:
                robot.avoid_obstacle(obstacle_pos)
                robot.previous_neighbor_count = current_neighbors
                continue

            # Otherwise, just continue forward (default state)
            robot.previous_neighbor_count = current_neighbors

        # 3. Move all robots
        for robot in self.robots:
            robot.move()

        self.current_step += 1

    def run(self):
        """Run the full simulation"""
        # initial metric/state recording (step 0)
        if self.enable_plotting:
            # record initial plot state
            self._record_state(0)
        if self.track_metrics:
            self._record_metrics()

        for step in range(1, self.max_steps + 1):
            self.step()

            if self.enable_plotting and step % self.plot_interval == 0:
                self._record_state(step)

            if self.track_metrics:
                self._record_metrics()

            if step % 100 == 0:
                print(f"  Completed step {step}/{self.max_steps}")

        if self.enable_plotting:
            print(f"Generating plot with {len(self.position_history)} time steps...")
            # Save single combined subplot figure as requested
            self._plot_all_states(output_filename=os.path.join(self.plot_output_dir, 'part_a_plots.png'))
            print(f"Plot saved as '{os.path.join(self.plot_output_dir, 'part_a_plots.png')}'")

    def _record_state(self, step: int):
        """Record the current state of all robots for later plotting"""
        robot_data = []
        for robot in self.robots:
            robot_data.append({
                'id': robot.id,
                'position': robot.position.copy(),
                'direction': robot.direction.copy(),
                'is_illuminated': robot.is_illuminated,
                'avoidance_radius': robot.current_avoidance_radius
            })
        self.position_history.append((step, robot_data))

    def _plot_all_states(self, output_filename: str = 'part_a_plots.png'):
        """Create a single plot showing robot positions at all recorded time steps (multiple subplots in one image)"""
        if len(self.position_history) == 0:
            print("No states recorded to plot.")
            return

        num_plots = len(self.position_history)
        cols = 5
        rows = (num_plots + cols - 1) // cols

        fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows))
        # ensure axes is a flat 1D array
        axes = np.array(axes).reshape(-1)

        for idx, (step, robot_data) in enumerate(self.position_history):
            ax = axes[idx]
            ax.set_xlim(0, self.world_size[0])
            ax.set_ylim(0, self.world_size[1])
            ax.set_aspect('equal')
            ax.grid(True, alpha=0.3)
            ax.set_title(f'Step {step}', fontsize=10, fontweight='bold')

            beacon_circle = Circle(self.beacon.position, 0.3, color='yellow',
                                   ec='orange', linewidth=2, zorder=5)
            ax.add_patch(beacon_circle)

            for robot_state in robot_data:
                color = 'green' if robot_state['is_illuminated'] else 'red'
                alpha = 0.1

                avoidance_circle = Circle(robot_state['position'],
                                         robot_state['avoidance_radius'],
                                         color=color, alpha=alpha, zorder=1)
                ax.add_patch(avoidance_circle)

                robot_circle = Circle(robot_state['position'], 0.3,
                                      color=color, ec='black', linewidth=1.5,
                                      alpha=0.7, zorder=3)
                ax.add_patch(robot_circle)

                arrow_length = 0.6
                dx = robot_state['direction'][0] * arrow_length
                dy = robot_state['direction'][1] * arrow_length
                ax.arrow(robot_state['position'][0], robot_state['position'][1],
                         dx, dy, head_width=0.2, head_length=0.15,
                         fc='black', ec='black', zorder=4, linewidth=1.5)

        # turn off empty subplots
        for idx in range(num_plots, len(axes)):
            axes[idx].axis('off')

        legend_elements = [
            Patch(facecolor='green', alpha=0.7, label='Illuminated'),
            Patch(facecolor='red', alpha=0.7, label='Shadowed'),
            Line2D([0], [0], marker='*', color='orange', label='Beacon',
                   markersize=12, linestyle='None')
        ]
        fig.legend(handles=legend_elements, loc='upper center',
                   ncol=3, fontsize=12, frameon=True)

        plt.suptitle(f'Emergent Taxis Simulation - N={self.num_robots}, Steps=0-{self.max_steps}, Interval={self.plot_interval}',
                     fontsize=14, fontweight='bold', y=0.995)
        plt.tight_layout(rect=[0, 0, 1, 0.96])

        os.makedirs(os.path.dirname(output_filename) or '.', exist_ok=True)
        plt.savefig(output_filename, dpi=150, bbox_inches='tight')
        plt.close()
        print(f"  Saved: {output_filename}")

    @staticmethod
    def run_multiple_experiments(num_runs: int = 20, num_robots: int = 10,
                                max_steps: int = 1000, world_size: Tuple[float, float] = (20, 20),
                                plot_output_dir: str = 'plots'):
        """Run multiple independent experiments and plot aggregate results (Part B)"""
        print("=" * 60)
        print("EMERGENT TAXIS - MULTIPLE EXPERIMENTS")
        print("=" * 60)
        print(f"Parameters:")
        print(f"  - Number of runs: {num_runs}")
        print(f"  - Number of robots: {num_robots}")
        print(f"  - Maximum steps: {max_steps}")
        print("=" * 60)
        print("\nRunning experiments...")

        all_distances = []
        all_cohesions = []

        for run_id in range(num_runs):
            print(f"  Run {run_id + 1}/{num_runs}...")

            # Create simulation with metrics tracking
            sim = Simulation(
                num_robots=num_robots,
                max_steps=max_steps,
                world_size=world_size,
                initialization_mode='connected',
                enable_plotting=False,
                track_metrics=True
            )

            # Run the simulation (this will track metrics automatically)
            sim.run()

            all_distances.append(sim.distances_to_beacon)
            all_cohesions.append(sim.cohesion_values)

        print("\n" + "=" * 60)
        print("All experiments complete!")
        print("=" * 60)

        # Align lengths (some safety) -> truncate to shortest run if needed
        min_len = min(len(a) for a in all_distances) if len(all_distances) > 0 else 0
        if min_len == 0:
            raise RuntimeError("No metric data collected across runs.")
        all_distances = np.array([a[:min_len] for a in all_distances])
        all_cohesions = np.array([c[:min_len] for c in all_cohesions])

        # Calculate statistics
        mean_distances = np.mean(all_distances, axis=0)
        std_distances = np.std(all_distances, axis=0)
        mean_cohesions = np.mean(all_cohesions, axis=0)
        std_cohesions = np.std(all_cohesions, axis=0)

        # Create plots
        print("\nGenerating plots...")
        Simulation._plot_aggregate_results(
            mean_distances, std_distances,
            mean_cohesions, std_cohesions,
            max_steps, plot_output_dir
        )

        # Print statistics
        Simulation._print_statistics(mean_distances, mean_cohesions)

        return {
            'mean_distances': mean_distances,
            'std_distances': std_distances,
            'mean_cohesions': mean_cohesions,
            'std_cohesions': std_cohesions,
            'all_distances': all_distances,
            'all_cohesions': all_cohesions
        }

    @staticmethod
    def _plot_aggregate_results(mean_distances, std_distances, mean_cohesions, std_cohesions,
                                max_steps, output_dir):
        """Create plots for Part B analysis"""
        os.makedirs(output_dir, exist_ok=True)
        time_steps = np.arange(len(mean_distances))  # use actual length

        # Combined plot with both metrics
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

        # Distance to beacon
        ax1.plot(time_steps, mean_distances, '-', linewidth=2, label='Mean distance')
        ax1.fill_between(time_steps,
                         mean_distances - std_distances,
                         mean_distances + std_distances,
                         alpha=0.3, color='blue', label='±1 std dev')
        ax1.set_xlabel('Time Steps', fontsize=12)
        ax1.set_ylabel('Distance to Beacon', fontsize=12)
        ax1.set_title('Average Distance of Swarm Centroid to Beacon Over Time',
                      fontsize=14, fontweight='bold')
        ax1.grid(True, alpha=0.3)
        ax1.legend(fontsize=10)

        # Cohesion
        ax2.plot(time_steps, mean_cohesions, '-', linewidth=2, label='Mean cohesion')
        ax2.fill_between(time_steps,
                         mean_cohesions - std_cohesions,
                         mean_cohesions + std_cohesions,
                         alpha=0.3, color='green', label='±1 std dev')
        ax2.set_xlabel('Time Steps', fontsize=12)
        ax2.set_ylabel('Cohesion (Avg Distance to Centroid)', fontsize=12)
        ax2.set_title('Swarm Cohesion Over Time', fontsize=14, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        ax2.legend(fontsize=10)

        plt.tight_layout()
        filename = os.path.join(output_dir, 'part_b_analysis.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight')
        plt.close()
        print(f"  Saved: {filename}")

        # Individual plots: Distance
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.plot(time_steps, mean_distances, '-', linewidth=2, label='Mean distance')
        ax.fill_between(time_steps,
                        mean_distances - std_distances,
                        mean_distances + std_distances,
                        alpha=0.3, color='blue', label='±1 std dev')
        ax.set_xlabel('Time Steps', fontsize=12)
        ax.set_ylabel('Distance to Beacon', fontsize=12)
        ax.set_title('Average Distance of Swarm Centroid to Beacon Over Time',
                     fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.legend(fontsize=10)
        plt.tight_layout()
        filename = os.path.join(output_dir, 'part_b_distance_to_beacon.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight')
        plt.close()
        print(f"  Saved: {filename}")

        # Individual plots: Cohesion
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.plot(time_steps, mean_cohesions, '-', linewidth=2, label='Mean cohesion')
        ax.fill_between(time_steps,
                        mean_cohesions - std_cohesions,
                        mean_cohesions + std_cohesions,
                        alpha=0.3, color='green', label='±1 std dev')
        ax.set_xlabel('Time Steps', fontsize=12)
        ax.set_ylabel('Cohesion (Avg Distance to Centroid)', fontsize=12)
        ax.set_title('Swarm Cohesion Over Time', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.legend(fontsize=10)
        plt.tight_layout()
        filename = os.path.join(output_dir, 'part_b_cohesion.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight')
        plt.close()
        print(f"  Saved: {filename}")

    @staticmethod
    def _print_statistics(mean_distances, mean_cohesions):
        """Print summary statistics"""
        print("\n" + "=" * 60)
        print("SUMMARY STATISTICS")
        print("=" * 60)

        initial_distance = mean_distances[0]
        final_distance = mean_distances[-1]
        distance_reduction = initial_distance - final_distance

        print(f"\nDistance to Beacon:")
        print(f"  Initial (step 0): {initial_distance:.4f} units")
        print(f"  Final (last): {final_distance:.4f} units")
        print(f"  Reduction: {distance_reduction:.4f} units ({(distance_reduction / initial_distance * 100) if initial_distance != 0 else 0:.2f}%)")

        initial_cohesion = mean_cohesions[0]
        final_cohesion = mean_cohesions[-1]

        print(f"\nCohesion:")
        print(f"  Initial (step 0): {initial_cohesion:.4f} units")
        print(f"  Final (last): {final_cohesion:.4f} units")
        print(f"  Change: {final_cohesion - initial_cohesion:.4f} units")

        print("=" * 60)

    @staticmethod
    def run_parameter_sweep(rai_values: List[float], ras_values: List[float],
                           num_runs: int = 20, num_robots: int = 10,
                           max_steps: int = 1000, world_size: Tuple[float, float] = (20, 20),
                           plot_output_dir: str = 'plots'):
        """Run experiments with different avoidance radii and analyze results (Part C)"""
        print("=" * 60)
        print("EMERGENT TAXIS - PARAMETER SWEEP")
        print("=" * 60)
        print(f"Parameters:")
        print(f"  - Number of runs per configuration: {num_runs}")
        print(f"  - Number of robots: {num_robots}")
        print(f"  - Maximum steps: {max_steps}")
        print(f"  - rai values: {rai_values}")
        print(f"  - ras values: {ras_values}")
        print("=" * 60)

        # Storage for results
        results_distance = {}  # Key: (rai, ras), Value: list of final distances
        results_cohesion = {}  # Key: (rai, ras), Value: list of final cohesions

        total_configs = len(rai_values) * len(ras_values)
        current_config = 0

        # Test each combination of rai and ras
        for rai in rai_values:
            for ras in ras_values:
                current_config += 1
                print(f"\n[{current_config}/{total_configs}] Testing rai={rai:.2f}, ras={ras:.2f}")

                # Only test if rai > ras (illuminated should have larger radius)
                if rai <= ras:
                    print(f"  Skipping: rai must be > ras")
                    continue

                final_distances = []
                final_cohesions = []

                for run_id in range(num_runs):
                    # Create simulation with custom avoidance radii
                    sim = Simulation(
                        num_robots=num_robots,
                        max_steps=max_steps,
                        world_size=world_size,
                        initialization_mode='connected',
                        enable_plotting=False,
                        track_metrics=True
                    )

                    # Override the avoidance radii for all robots
                    for robot in sim.robots:
                        robot.avoidance_radius_illuminated = rai
                        robot.avoidance_radius_shadowed = ras
                        robot.current_avoidance_radius = rai if robot.is_illuminated else ras

                    # Run simulation using the run() method which tracks metrics
                    sim.run()

                    # Store final values (safely)
                    if len(sim.distances_to_beacon) > 0:
                        final_distances.append(sim.distances_to_beacon[-1])
                    else:
                        final_distances.append(np.nan)
                    if len(sim.cohesion_values) > 0:
                        final_cohesions.append(sim.cohesion_values[-1])
                    else:
                        final_cohesions.append(np.nan)

                results_distance[(rai, ras)] = final_distances
                results_cohesion[(rai, ras)] = final_cohesions

                mean_dist = np.nanmean(final_distances)
                mean_coh = np.nanmean(final_cohesions)
                print(f"  Mean final distance: {mean_dist:.4f}, Mean final cohesion: {mean_coh:.4f}")

        print("\n" + "=" * 60)
        print("Parameter sweep complete!")
        print("=" * 60)

        # Create plots
        print("\nGenerating plots...")
        Simulation._plot_parameter_sweep_results(
            results_distance, results_cohesion,
            rai_values, ras_values, plot_output_dir
        )

        return results_distance, results_cohesion

    @staticmethod
    def _plot_parameter_sweep_results(results_distance, results_cohesion,
                                      rai_values, ras_values, output_dir):
        """Create heatmaps and plots for parameter sweep results"""
        os.makedirs(output_dir, exist_ok=True)

        # Prepare data for heatmaps
        distance_matrix = np.full((len(rai_values), len(ras_values)), np.nan)
        cohesion_matrix = np.full((len(rai_values), len(ras_values)), np.nan)

        for i, rai in enumerate(rai_values):
            for j, ras in enumerate(ras_values):
                if (rai, ras) in results_distance:
                    distance_matrix[i, j] = np.nanmean(results_distance[(rai, ras)])
                    cohesion_matrix[i, j] = np.nanmean(results_cohesion[(rai, ras)])

        # Create figure with two heatmaps
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

        # Heatmap 1: Final distance to beacon
        im1 = ax1.imshow(distance_matrix, cmap='RdYlGn_r', aspect='auto')
        ax1.set_xticks(np.arange(len(ras_values)))
        ax1.set_yticks(np.arange(len(rai_values)))
        ax1.set_xticklabels([f'{v:.2f}' for v in ras_values])
        ax1.set_yticklabels([f'{v:.2f}' for v in rai_values])
        ax1.set_xlabel('ras (Shadowed Avoidance Radius)', fontsize=12)
        ax1.set_ylabel('rai (Illuminated Avoidance Radius)', fontsize=12)
        ax1.set_title('Final Distance to Beacon After Simulation', fontsize=14, fontweight='bold')

        # Add values to cells
        for i in range(len(rai_values)):
            for j in range(len(ras_values)):
                if not np.isnan(distance_matrix[i, j]):
                    ax1.text(j, i, f'{distance_matrix[i, j]:.2f}',
                             ha="center", va="center", color="black", fontsize=9)

        cbar1 = plt.colorbar(im1, ax=ax1)
        cbar1.set_label('Distance to Beacon', fontsize=11)

        # Heatmap 2: Final cohesion
        im2 = ax2.imshow(cohesion_matrix, cmap='RdYlGn', aspect='auto')
        ax2.set_xticks(np.arange(len(ras_values)))
        ax2.set_yticks(np.arange(len(rai_values)))
        ax2.set_xticklabels([f'{v:.2f}' for v in ras_values])
        ax2.set_yticklabels([f'{v:.2f}' for v in rai_values])
        ax2.set_xlabel('ras (Shadowed Avoidance Radius)', fontsize=12)
        ax2.set_ylabel('rai (Illuminated Avoidance Radius)', fontsize=12)
        ax2.set_title('Final Cohesion After Simulation', fontsize=14, fontweight='bold')

        # Add values to cells
        for i in range(len(rai_values)):
            for j in range(len(ras_values)):
                if not np.isnan(cohesion_matrix[i, j]):
                    ax2.text(j, i, f'{cohesion_matrix[i, j]:.2f}',
                             ha="center", va="center", color="black", fontsize=9)

        cbar2 = plt.colorbar(im2, ax=ax2)
        cbar2.set_label('Cohesion', fontsize=11)

        plt.tight_layout()
        filename = os.path.join(output_dir, 'part_c_parameter_sweep.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight')
        plt.close()
        print(f"   Saved: {filename}")

        # Also create line plots showing the effect of varying each parameter
        Simulation._plot_parameter_effects(results_distance, results_cohesion,
                                          rai_values, ras_values, output_dir)

    @staticmethod
    def _plot_parameter_effects(results_distance, results_cohesion,
                                rai_values, ras_values, output_dir):
        """Create line plots showing effect of varying each parameter"""
        os.makedirs(output_dir, exist_ok=True)

        # Plot 1: Effect of varying rai (fix ras at middle value)
        middle_ras = ras_values[len(ras_values) // 2]
        rai_distances = []
        rai_cohesions = []

        for rai in rai_values:
            if (rai, middle_ras) in results_distance:
                rai_distances.append(np.nanmean(results_distance[(rai, middle_ras)]))
                rai_cohesions.append(np.nanmean(results_cohesion[(rai, middle_ras)]))
            else:
                rai_distances.append(np.nan)
                rai_cohesions.append(np.nan)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        ax1.plot(rai_values, rai_distances, 'bo-', linewidth=2, markersize=8)
        ax1.set_xlabel('rai (Illuminated Avoidance Radius)', fontsize=12)
        ax1.set_ylabel('Final Distance to Beacon', fontsize=12)
        ax1.set_title(f'Effect of rai on Distance (ras={middle_ras:.2f})', fontsize=13, fontweight='bold')
        ax1.grid(True, alpha=0.3)

        ax2.plot(rai_values, rai_cohesions, 'go-', linewidth=2, markersize=8)
        ax2.set_xlabel('rai (Illuminated Avoidance Radius)', fontsize=12)
        ax2.set_ylabel('Final Cohesion', fontsize=12)
        ax2.set_title(f'Effect of rai on Cohesion (ras={middle_ras:.2f})', fontsize=13, fontweight='bold')
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()
        filename = os.path.join(output_dir, 'part_c_rai_effect.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight')
        plt.close()
        print(f"  Saved: {filename}")

        # Plot 2: Effect of varying ras (fix rai at middle value)
        middle_rai = rai_values[len(rai_values) // 2]
        ras_distances = []
        ras_cohesions = []

        for ras in ras_values:
            if (middle_rai, ras) in results_distance:
                ras_distances.append(np.nanmean(results_distance[(middle_rai, ras)]))
                ras_cohesions.append(np.nanmean(results_cohesion[(middle_rai, ras)]))
            else:
                ras_distances.append(np.nan)
                ras_cohesions.append(np.nan)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        ax1.plot(ras_values, ras_distances, 'bo-', linewidth=2, markersize=8)
        ax1.set_xlabel('ras (Shadowed Avoidance Radius)', fontsize=12)
        ax1.set_ylabel('Final Distance to Beacon', fontsize=12)
        ax1.set_title(f'Effect of ras on Distance (rai={middle_rai:.2f})', fontsize=13, fontweight='bold')
        ax1.grid(True, alpha=0.3)

        ax2.plot(ras_values, ras_cohesions, 'go-', linewidth=2, markersize=8)
        ax2.set_xlabel('ras (Shadowed Avoidance Radius)', fontsize=12)
        ax2.set_ylabel('Final Cohesion', fontsize=12)
        ax2.set_title(f'Effect of ras on Cohesion (rai={middle_rai:.2f})', fontsize=13, fontweight='bold')
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()
        filename = os.path.join(output_dir, 'part_c_ras_effect.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight')
        plt.close()
        print(f"  Saved: {filename}")


# a)

Implement emergent taxis and execute a run with N = 10 robots and 1,000 time steps. Every
5 time steps, plot the positions of all robots in 2D space. Show the illumination status and
avoidance radius for each robot in the same plot.
You are free to use a programming language of your choice, but please include a guide how
to execute your code. You may model the beacon as an area light source with parallel rays of
light. Make sure that all robots are initialized fully connected.

In [27]:

def main():
    """Main function for Part A - Emergent Taxis Simulation"""
    
    print("=" * 60)
    print("EMERGENT TAXIS SIMULATION - PART A")
    print("=" * 60)
    print("Parameters:")
    print("  - Number of robots (N): 10")
    print("  - Maximum steps: 1,000")
    print("  - Plot interval: Every 5 steps")
    print("  - World size: 20 × 20 units")
    print("  - Initialization: Fully connected cluster")
    print("  - Robot starting position: Left side (20% from left)")
    print("  - Beacon position: Right side (80%, center)")
    print("  - Algorithm: As per Bjerknes et al. (2007)")
    print("=" * 60)
    
    # Create and run simulation with plotting enabled
    sim = Simulation(
        num_robots=10,
        max_steps=1000,
        world_size=(20, 20),
        initialization_mode='connected',
        enable_plotting=True,
        plot_interval=5,
        plot_output_dir='plots'
    )
    
    # Display initial configuration
    print("\nInitial robot positions:")
    for i, robot in enumerate(sim.robots):
        print(f"  Robot {i}: pos=({robot.position[0]:.2f}, {robot.position[1]:.2f}), "
              f"dir=({robot.direction[0]:.2f}, {robot.direction[1]:.2f})")
    
    print(f"\nBeacon position: ({sim.beacon.position[0]:.2f}, {sim.beacon.position[1]:.2f})")
    
    # Run the simulation
    sim.run()
    
    print("\n" + "=" * 60)
    print("SIMULATION SUMMARY")
    print("=" * 60)
    print(f"Total time steps executed: {sim.current_step}")
    print(f"Total snapshots recorded: {len(sim.position_history)}")
    print(f"Output file: plots/emergent_taxis_simulation.png")
    print("=" * 60)


if __name__ == "__main__":
    main()

EMERGENT TAXIS SIMULATION - PART A
Parameters:
  - Number of robots (N): 10
  - Maximum steps: 1,000
  - Plot interval: Every 5 steps
  - World size: 20 × 20 units
  - Initialization: Fully connected cluster
  - Robot starting position: Left side (20% from left)
  - Beacon position: Right side (80%, center)
  - Algorithm: As per Bjerknes et al. (2007)

Verifying connectivity...
  Robot 0: 9 neighbors
  Robot 1: 9 neighbors
  Robot 2: 9 neighbors
  Robot 3: 9 neighbors
  Robot 4: 9 neighbors
  Robot 5: 9 neighbors
  Robot 6: 9 neighbors
  Robot 7: 9 neighbors
  Robot 8: 9 neighbors
  Robot 9: 9 neighbors
✓ All robots are fully connected!

✓ Plot output directory created: 'plots/'

Initial robot positions:
  Robot 0: pos=(4.20, 10.19), dir=(-0.78, -0.62)
  Robot 1: pos=(3.88, 10.95), dir=(-0.62, 0.78)
  Robot 2: pos=(4.12, 10.10), dir=(-0.98, 0.21)
  Robot 3: pos=(3.53, 10.04), dir=(0.03, 1.00)
  Robot 4: pos=(3.96, 10.91), dir=(-0.76, -0.65)
  Robot 5: pos=(4.55, 10.69), dir=(-0.60, 0.8

# b)

Plot the average distance of the swarm to the light source and the cohesion of the swarm over
time. The cohesion is the average distance of the robots to the centroid of the swarm. The
centroid is the average X and Y position of all robots. Execute at least 20 independent runs.

In [28]:
def main():
    """Main function for Part B - Multiple Experiments Analysis"""
    
    # Run 20 independent experiments and generate plots
    Simulation.run_multiple_experiments(
        num_runs=20,
        num_robots=10,
        max_steps=1000,
        world_size=(20, 20),
        plot_output_dir='plots'
    )


if __name__ == "__main__":
    main()

EMERGENT TAXIS - MULTIPLE EXPERIMENTS
Parameters:
  - Number of runs: 20
  - Number of robots: 10
  - Maximum steps: 1000

Running experiments...
  Run 1/20...

Verifying connectivity...
  Robot 0: 9 neighbors
  Robot 1: 9 neighbors
  Robot 2: 9 neighbors
  Robot 3: 9 neighbors
  Robot 4: 9 neighbors
  Robot 5: 9 neighbors
  Robot 6: 9 neighbors
  Robot 7: 9 neighbors
  Robot 8: 9 neighbors
  Robot 9: 9 neighbors
✓ All robots are fully connected!
  Completed step 100/1000
  Completed step 200/1000
  Completed step 300/1000
  Completed step 400/1000
  Completed step 500/1000
  Completed step 600/1000
  Completed step 700/1000
  Completed step 800/1000
  Completed step 900/1000
  Completed step 1000/1000
  Run 2/20...

Verifying connectivity...
  Robot 0: 9 neighbors
  Robot 1: 9 neighbors
  Robot 2: 9 neighbors
  Robot 3: 9 neighbors
  Robot 4: 9 neighbors
  Robot 5: 9 neighbors
  Robot 6: 9 neighbors
  Robot 7: 9 neighbors
  Robot 8: 9 neighbors
  Robot 9: 9 neighbors
✓ All robots are 

# c)

Plot the average distance of the swarm to the light source and the cohesion of the swarm after
1,000 time steps for varied values of rai and ras. Show the influence of the different values on
the behavior of the swarm.

In [26]:
def main():
    """Main function for Part C - Parameter Variation Analysis"""
    
    # Define parameter ranges to test
    # We'll vary both illuminated and shadowed avoidance radii
    rai_values = [0.4, 0.51, 0.7]   # low, mid, high illuminated avoidance radius
    ras_values = [0.3, 0.4, 0.5]    # low, mid, high shadowed avoidance radius

    
    # Run experiments with different parameter combinations
    Simulation.run_parameter_sweep(
        rai_values=rai_values,
        ras_values=ras_values,
        num_runs=5,
        num_robots=10,
        max_steps=1000,
        world_size=(20, 20),
        plot_output_dir='plots'
    )


if __name__ == "__main__":
    main()

EMERGENT TAXIS - PARAMETER SWEEP
Parameters:
  - Number of runs per configuration: 5
  - Number of robots: 10
  - Maximum steps: 1000
  - rai values: [0.4, 0.51, 0.7]
  - ras values: [0.3, 0.4, 0.5]

[1/9] Testing rai=0.40, ras=0.30

Verifying connectivity...
  Robot 0: 9 neighbors
  Robot 1: 9 neighbors
  Robot 2: 9 neighbors
  Robot 3: 9 neighbors
  Robot 4: 9 neighbors
  Robot 5: 9 neighbors
  Robot 6: 9 neighbors
  Robot 7: 9 neighbors
  Robot 8: 9 neighbors
  Robot 9: 9 neighbors
✓ All robots are fully connected!
  Completed step 100/1000
  Completed step 200/1000
  Completed step 300/1000
  Completed step 400/1000
  Completed step 500/1000
  Completed step 600/1000
  Completed step 700/1000
  Completed step 800/1000
  Completed step 900/1000
  Completed step 1000/1000

Verifying connectivity...
  Robot 0: 9 neighbors
  Robot 1: 9 neighbors
  Robot 2: 9 neighbors
  Robot 3: 9 neighbors
  Robot 4: 9 neighbors
  Robot 5: 9 neighbors
  Robot 6: 9 neighbors
  Robot 7: 9 neighbors
  Ro